# Case 1: Italian daily streamflow data download 


Italian database only shows the data for the most recent year, and not the full database by default.

Link to ISPRA: http://www.hiscentral.isprambiente.gov.it/hiscentral/hydromap.aspx?map=obsclient

# Import modules

In [15]:
import geopandas as gpd
from shapely.geometry import Point, Polygon
from plotly.offline import plot
import contextily as cx
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from mpl_toolkits.axes_grid1 import make_axes_locatable
import warnings
import datetime
import tqdm as tqdm
import os
import glob
warnings.simplefilter(action='ignore', category=Warning)


%config Completer.use_jedi = False # This improve the auto-fill function

In [16]:
path = r"C:\Users\nascimth\Documents\Thiago\Eawag\Python\Scripts\Data\Streamflow"
os.chdir(path)
os.getcwd()

'C:\\Users\\nascimth\\Documents\\Thiago\\Eawag\\Python\\Scripts\\Data\\Streamflow'

In [17]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
import html5lib

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

# import Action chains 
from selenium.webdriver.common.action_chains import ActionChains
import time

# List of stations

In [18]:
list_stations = pd.read_excel(r'C:\Users\nascimth\Documents\Thiago\Eawag\Python\Data\Streamflow\IT\original\ISPRA\network_ISPRA.xlsx')
list_stations

,Code
0,hsl-abr:5010
1,hsl-abr:5120
2,hsl-abr:5210
3,hsl-abr:5230
4,hsl-abr:5320
...,...
509,hsl-vda:1570
510,hsl-vda:1640
511,hsl-vda:1650
512,hsl-vda:1660


In [20]:
# Import necessary libraries
import requests
import xml.etree.ElementTree as ET
import pandas as pd
from tqdm import tqdm

# Iterate over a list of station codes (modify the range as needed)
for station in tqdm(list_stations.Code[0:2]):
    # Assign the current station code to the site_name variable
    site_name = station
    
    # Define the file name for saving the CSV file, using a placeholder "site_name"
    # This will be replaced later with the actual site name
    file_name = r"C:\Users\nascimth\Documents\Thiago\Eawag\Python\Data\Streamflow\IT\original\ISPRA\original\site_name.csv"
    
    # Define the URL for fetching data, with placeholders for "STATION" and "REGION"
    URL = "http://hydroserver.ddns.net/italia/REGION/index.php/default/services/cuahsi_1_1.asmx/GetValuesObject?authToken=&location=STATION&variable=REGION:Discharge"
    
    # Replace the placeholders in the URL with the current site_name
    url = URL.replace("STATION", site_name)
    url = url.replace("REGION", site_name[0:7])
    
    # Send an HTTP GET request to the URL
    response = requests.get(url)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Parse the XML content received in the response
        root = ET.fromstring(response.text)

        # Initialize lists to store data extracted from the XML
        date_times = []
        values = []
        quality_control_levels = []

        # Iterate through the 'value' elements in the XML and extract relevant data
        for value_elem in root.findall('.//{http://www.cuahsi.org/waterML/1.1/}value'):
            date_time = value_elem.get('dateTime')
            value = float(value_elem.text)
            quality_control_level = int(value_elem.get('qualityControlLevelCode'))
            date_times.append(date_time)
            values.append(value)
            quality_control_levels.append(quality_control_level)

        # Create a DataFrame from the extracted data
        df = pd.DataFrame({'dateTime': date_times, 'Value': values, 'QualityControlLevelCode': quality_control_levels})
        
        # Replace the "site_name" placeholder in the file name with the actual site_name,
        # and replace colons with underscores in the site_name for a valid file name
        file_name = file_name.replace("site_name", site_name.replace(":", "_"))

        # Save the DataFrame as a CSV file with the modified file name
        df.to_csv(file_name, index=False)

    else:
        # Print an error message if the HTTP request was not successful
        print(f"Failed to fetch data. Status code: {response.status_code}")


100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:01<00:00,  1.63it/s]


In [21]:
df

,dateTime,Value,QualityControlLevelCode
0,2002-01-01T00:00:00,0.84,1
1,2002-01-02T00:00:00,0.57,1
2,2002-01-03T00:00:00,0.57,1
3,2002-01-04T00:00:00,0.49,1
4,2002-01-05T00:00:00,0.40,1
...,...,...,...
725,2003-12-27T00:00:00,0.80,1
726,2003-12-28T00:00:00,0.80,1
727,2003-12-29T00:00:00,0.80,1
728,2003-12-30T00:00:00,0.80,1


In [19]:
# Import necessary libraries
import requests
import xml.etree.ElementTree as ET
import pandas as pd
from tqdm import tqdm

# Define an empty list to store data for each station
all_station_data = []
# Iterate over a list of station codes (modify the range as needed)
for station in tqdm(list_stations.Code[0:2]):
    # Assign the current station code to the site_name variable
    site_name = station
    
    # Define the file name for saving the CSV file, using a placeholder "site_name"
    # This will be replaced later with the actual site name
    file_name = r"C:\Users\nascimth\Documents\Thiago\Eawag\Python\Data\Streamflow\IT\original\ISPRA\network_ISPRA_infos.xlsx"
    
    # Define the URL for fetching data, with placeholders for "STATION" and "REGION"
    URL = "http://hydroserver.ddns.net/italia/REGION/index.php/default/services/cuahsi_1_1.asmx/GetValuesObject?authToken=&location=STATION&variable=REGION:Discharge"
    
    # Replace the placeholders in the URL with the current site_name
    url = URL.replace("STATION", site_name)
    url = url.replace("REGION", site_name[0:7])
    
    # Send an HTTP GET request to the URL
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the XML content
        root = ET.fromstring(response.text)

        # Initialize lists to store data
        site_names = []
        site_codes = []
        site_ids = []
        latitudes = []
        longitudes = []

        # Iterate through the 'sourceInfo' elements and extract the desired information
        for source_info_elem in root.findall('.//{http://www.cuahsi.org/waterML/1.1/}sourceInfo'):
            site_name = source_info_elem.find('{http://www.cuahsi.org/waterML/1.1/}siteName').text
            site_code = source_info_elem.find('{http://www.cuahsi.org/waterML/1.1/}siteCode').text
            site_id = source_info_elem.find('{http://www.cuahsi.org/waterML/1.1/}siteCode').get('siteID')
            latitude = source_info_elem.find('.//{http://www.cuahsi.org/waterML/1.1/}latitude').text
            longitude = source_info_elem.find('.//{http://www.cuahsi.org/waterML/1.1/}longitude').text

            site_names.append(site_name)
            site_codes.append(site_code)
            site_ids.append(site_id)
            latitudes.append(latitude)
            longitudes.append(longitude)

        # Create a DataFrame for the extracted site information
        site_info_df = pd.DataFrame({
            'siteName': site_names,
            'code': station,
            'siteCode': site_codes,
            'siteID': site_ids,
            'latitude': latitudes,
            'longitude': longitudes
        })


        # Append the DataFrame to the list of DataFrames
        all_station_data.append(site_info_df)

    else:
        print(f"Failed to fetch data. Status code: {response.status_code}")

# Concatenate all DataFrames row-wise to create a single DataFrame
final_df = pd.concat(all_station_data, ignore_index=True)

# Print the final DataFrame
final_df

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:01<00:00,  1.76it/s]


,siteName,code,siteCode,siteID,latitude,longitude
0,VIBRATA ad ALBA ADRIATICA,hsl-abr:5010,5010,2138,42.83583333,13.91916667
1,SALINELLO a CAVATASSI,hsl-abr:5120,5120,2141,42.78388889,13.90583333


# Case 2: French daily streamflow data download 


France does not provide one simple way to download all data at once, hence we can work with selenium to automatize the process. There are more than 3000 stations.

Link to the Eau France and Hydroportal: https://www.hydro.eaufrance.fr/rechercher/entites-hydrometriques

# Import modules

In [2]:
import geopandas as gpd
from shapely.geometry import Point, Polygon
from plotly.offline import plot
import contextily as cx
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from mpl_toolkits.axes_grid1 import make_axes_locatable
import warnings
import datetime
import tqdm as tqdm
import os
import glob
warnings.simplefilter(action='ignore', category=Warning)


%config Completer.use_jedi = False # This improve the auto-fill function

# Network organization

1. First we need to have a look at the dataset and select with a filter select the lsit of stations to be downloaded:

In [3]:
network = pd.read_csv(r'C:\Users\nascimth\Documents\Thiago\Eawag\Python\Data\Streamflow\France\stations\stationshydrofrance_all.csv',encoding='latin-1', skiprows=0, usecols = ['CdStationH', 'CdAncieRef', 
                      'TypStation', 'timePositi', 'DtFermetur', 'xcoord',
                      'ycoord'])


names = ['Code', 'CodeOld', 'Type', 'StartDate', 'EndDate', 'xcoord', 'ycoord']

network.columns = names


network["StartDate"].replace({'T': ' '}, regex=True, inplace=True)
network["StartDate"].replace({'Z': ' '}, regex=True, inplace=True)
network["EndDate"].replace({'T': ' '}, regex=True, inplace=True)
network["EndDate"].replace({'Z': ' '}, regex=True, inplace=True)

network["StartDate"] = pd.to_datetime(network['StartDate'], format='%Y/%m/%d')
network["EndDate"] = pd.to_datetime(network['EndDate'], format='%Y/%m/%d')

network

,Code,CodeOld,Type,StartDate,EndDate,xcoord,ycoord
0,A021005050,A0210050,STD,1899-12-30 23:50:39,NaT,7.606153,47.556970
1,A022020001,A0220200,STD,1993-01-24 23:00:00,NaT,7.528982,47.564377
2,A022065001,A0220650,STD,1993-01-24 23:00:00,1996-04-12 10:00:00,7.508168,47.587489
3,A023010001,A0220570,STD,1993-04-01 22:00:00,1996-04-12 10:00:00,7.545294,47.603575
4,A023020001,A0230201,STD,1993-02-03 23:00:00,1996-04-12 10:00:00,7.556488,47.596980
...,...,...,...,...,...,...,...
5175,Y980000301,Y9806210,STD,1971-05-31 23:00:00,1989-04-01 10:00:00,9.237134,41.596813
5176,Y980000302,Y9800003,STD,2020-06-16 22:00:00,NaT,9.237163,41.596406
5177,Y982000101,Y9825010,STD,1968-12-31 23:00:00,1980-12-01 11:00:00,9.204303,41.444451
5178,Y991000101,Y9905010,STD,1983-11-15 23:00:00,1988-04-30 22:00:00,9.146374,41.464203


In [4]:
# The filter application is not a good idea:
network_cond_3 = network.copy()
network_cond_3

,Code,CodeOld,Type,StartDate,EndDate,xcoord,ycoord
0,A021005050,A0210050,STD,1899-12-30 23:50:39,NaT,7.606153,47.556970
1,A022020001,A0220200,STD,1993-01-24 23:00:00,NaT,7.528982,47.564377
2,A022065001,A0220650,STD,1993-01-24 23:00:00,1996-04-12 10:00:00,7.508168,47.587489
3,A023010001,A0220570,STD,1993-04-01 22:00:00,1996-04-12 10:00:00,7.545294,47.603575
4,A023020001,A0230201,STD,1993-02-03 23:00:00,1996-04-12 10:00:00,7.556488,47.596980
...,...,...,...,...,...,...,...
5175,Y980000301,Y9806210,STD,1971-05-31 23:00:00,1989-04-01 10:00:00,9.237134,41.596813
5176,Y980000302,Y9800003,STD,2020-06-16 22:00:00,NaT,9.237163,41.596406
5177,Y982000101,Y9825010,STD,1968-12-31 23:00:00,1980-12-01 11:00:00,9.204303,41.444451
5178,Y991000101,Y9905010,STD,1983-11-15 23:00:00,1988-04-30 22:00:00,9.146374,41.464203


In [5]:
final_stations = network_cond_3.Code.tolist()

# Web-scraping

In [6]:
# Set the directory:
path = r"C:\Users\nascimth\Documents\Thiago\Eawag\Python\Scripts\Data\Streamflow"
os.chdir(path)
os.getcwd()

'C:\\Users\\nascimth\\Documents\\Thiago\\Eawag\\Python\\Scripts\\Data\\Streamflow'

In [7]:
# Import modules:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd

In [8]:
# With this URL you go directly to the station website. You just need to replace "station_name" by theirs.
URL = "https://www.hydro.eaufrance.fr/stationhydro/station_name/series"

# É necessário ter o arquivo webdriver num diretório conhecido, ou no diretório corrente: 
driver = webdriver.Chrome()
for station in tqdm.tqdm(final_stations[0:2]):
    path = URL.replace("station_name", station)
    
    # First we get the URL:
    driver.get(path)
    
    # Error handling:
    # There is the possibility of having stations classified as deprecated stations and then you have to do an extra step 
    # to retrive it. To deal with that we have the try and except: 
    try:
        # First you set the starting and ending date:
        driver.find_element_by_xpath('//*[@id="hydro_series_startAt"]').send_keys("01/09/1800")
        driver.find_element_by_xpath('//*[@id="hydro_series_endAt"]').send_keys("31/12/2023")
        
        # This step is just for refresh:
        driver.find_element_by_xpath('//*[@id="hydro_series_endAt"]').send_keys(Keys.RETURN)
    
        # Those following are other bottons needed to be clicked or ticked:
        driver.find_element_by_xpath('//*[@id="hydro_series_variableType_1"]').click()
        driver.find_element_by_xpath('//*[@id="hydro_series_dailyVariable"]/option[3]').click()
        driver.find_element_by_xpath('//*[@id="hydro_series_step"]').send_keys("1")

        # Now you click in the search bottom:
        driver.find_element_by_xpath('/html/body/div[2]/main/div/div[2]/div/div[1]/div[2]/div/form/div[5]/button').click()

        # As the data will need to be refreshed on the page you will need to set a longer wait than the default otherwise
        # the bottoms will not appear:
        driver.implicitly_wait(10)
        
        # These are bottoms for set the units, change to the table format, and finally donwload the data:
        button = driver.find_element_by_xpath("/html/body/div[2]/main/div/div[2]/div/div[2]/div/div/div[4]/div/div[1]/div/div[1]/div/button[1]")
        driver.execute_script("arguments[0].click();", button)

        button = driver.find_element_by_xpath("/html/body/div[2]/main/div/div[2]/div/div[2]/div/div/div[2]/div/div/a[2]")
        driver.execute_script("arguments[0].click();", button)

        button = driver.find_element_by_xpath("/html/body/div[2]/main/div/div[2]/div/div[2]/div/div/div[5]/div[1]/button")
        driver.execute_script("arguments[0].click();", button)
    
    except:
        
        try:
            # It will refresh the system with the option of look for old stations:
        
            driver.get("https://www.hydro.eaufrance.fr/rechercher/entites-hydrometriques")
            driver.find_element_by_xpath('//*[@id="shortcut-0"]').click()
            driver.find_element_by_xpath('//*[@id="hydro_entities_search_code"]').send_keys(station)
            driver.find_element_by_xpath('//*[@id="form-search"]/form/div[1]/div/div[4]/button[1]').click()
        
        
            # And now makes the search again after the refresh:
            driver.get(path)
        
            # First you set the starting and ending date:
            driver.find_element_by_xpath('//*[@id="hydro_series_startAt"]').send_keys("01/01/1800")
            driver.find_element_by_xpath('//*[@id="hydro_series_endAt"]').send_keys("31/12/2023")
        
            # This step is just for refresh:
            driver.find_element_by_xpath('//*[@id="hydro_series_endAt"]').send_keys(Keys.RETURN)
    
            # Those following are other bottons needed to be clicked or ticked:
            driver.find_element_by_xpath('//*[@id="hydro_series_variableType_1"]').click()
            driver.find_element_by_xpath('//*[@id="hydro_series_dailyVariable"]/option[3]').click()
            driver.find_element_by_xpath('//*[@id="hydro_series_step"]').send_keys("1")

            # Now you click in the search bottom:
            driver.find_element_by_xpath('/html/body/div[2]/main/div/div[2]/div/div[1]/div[2]/div/form/div[5]/button').click()

            # As the data will need to be refreshed on the page you will need to set a longer wait than the default otherwise
            # the bottoms will not appear:
            driver.implicitly_wait(10)
        
            # These are bottoms for set the units, change to the table format, and finally donwload the data:
            button = driver.find_element_by_xpath("/html/body/div[2]/main/div/div[2]/div/div[2]/div/div/div[4]/div/div[1]/div/div[1]/div/button[1]")
            driver.execute_script("arguments[0].click();", button)

            button = driver.find_element_by_xpath("/html/body/div[2]/main/div/div[2]/div/div[2]/div/div/div[2]/div/div/a[2]")
            driver.execute_script("arguments[0].click();", button)

            button = driver.find_element_by_xpath("/html/body/div[2]/main/div/div[2]/div/div[2]/div/div/div[5]/div[1]/button")
            driver.execute_script("arguments[0].click();", button)     
        
        
            # If it still keep giving an error, you can simply pass and not download because probably there is no 
            # information about this station in the system:
        except:
            pass

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:05<00:00,  2.91s/it]


#### Here we read our data:

In [9]:
# First the network information:
network_streams = pd.read_excel(r'C:\Users\nascimth\Documents\Thiago\Eawag\Python\Data\Streamflow\France\timeseries\network_FR.xlsx', index_col=1)
network_streams

,Unnamed: 0,CodeOld,Type,StartDate,EndDate,xcoord,ycoord,area,level,new_code
Code,,,,,,,,,,
A021005050,0,A0210050,STD,1899-12-30 23:50:39,NaN,7.606153,47.556970,35878.0,246.0,FR00001
A022020001,1,A0220200,STD,1993-01-24 23:00:00,NaN,7.528982,47.564377,15.0,277.0,FR00002
A022065001,2,A0220650,STD,1993-01-24 23:00:00,1996-04-12 10:00:00,7.508168,47.587489,8.2,274.0,FR00003
A023010001,3,A0220570,STD,1993-04-01 22:00:00,1996-04-12 10:00:00,7.545294,47.603575,NaN,248.0,FR00004
A023020001,4,A0230201,STD,1993-02-03 23:00:00,1996-04-12 10:00:00,7.556488,47.596980,NaN,247.0,FR00005
...,...,...,...,...,...,...,...,...,...,...
Y980000301,3153,Y9806210,STD,1971-05-31 23:00:00,1989-04-01 10:00:00,9.237134,41.596813,53.0,34.0,FR03154
Y980000302,3154,Y9800003,STD,2020-06-16 22:00:00,NaN,9.237163,41.596406,53.0,34.0,FR03155
Y982000101,3155,Y9825010,STD,1968-12-31 23:00:00,1980-12-01 11:00:00,9.204303,41.444451,29.0,4.0,FR03156


#### Location name retrieve:

Here we retrieve the area and levels from the selected gauges using selenium.

In [11]:
URL = "https://www.hydro.eaufrance.fr/sitehydro/station_name/fiche"


driver = webdriver.Chrome()
for station in tqdm.tqdm(network_streams.index[0:2]):
    path = URL.replace("station_name", station[0:-2])
    
    # First we get the URL:
    driver.get(path)
    
    network_streams.loc[station, "name"] = driver.find_element_by_xpath('//*[@id="main"]/main/div/div[2]/div/div[3]/div[2]/span[2]').text
    

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:01<00:00,  1.46it/s]


In [12]:
network_streams

,Unnamed: 0,CodeOld,Type,StartDate,EndDate,xcoord,ycoord,area,level,new_code,name
Code,,,,,,,,,,,
A021005050,0,A0210050,STD,1899-12-30 23:50:39,NaN,7.606153,47.556970,35878.0,246.0,FR00001,Le Rhin en Suisse [Bâle [Rheinhalle]]
A022020001,1,A0220200,STD,1993-01-24 23:00:00,NaN,7.528982,47.564377,15.0,277.0,FR00002,L'Augraben à Hégenheim
A022065001,2,A0220650,STD,1993-01-24 23:00:00,1996-04-12 10:00:00,7.508168,47.587489,8.2,274.0,FR00003,Le ruisseau le Liesbach à Blotzheim
A023010001,3,A0220570,STD,1993-04-01 22:00:00,1996-04-12 10:00:00,7.545294,47.603575,NaN,248.0,FR00004,[Le Liesbach] à Saint-Louis [Langenhaeuser]
A023020001,4,A0230201,STD,1993-02-03 23:00:00,1996-04-12 10:00:00,7.556488,47.596980,NaN,247.0,FR00005,Le ruisseau l'Augraben à Saint-Louis [Michelfe...
...,...,...,...,...,...,...,...,...,...,...,...
Y980000301,3153,Y9806210,STD,1971-05-31 23:00:00,1989-04-01 10:00:00,9.237134,41.596813,53.0,34.0,FR03154,NaN
Y980000302,3154,Y9800003,STD,2020-06-16 22:00:00,NaN,9.237163,41.596406,53.0,34.0,FR03155,NaN
Y982000101,3155,Y9825010,STD,1968-12-31 23:00:00,1980-12-01 11:00:00,9.204303,41.444451,29.0,4.0,FR03156,NaN


#### Area and level retrieve:

Here we retrieve the area and levels from the selected gauges using selenium.

In [13]:
URL = "https://www.hydro.eaufrance.fr/sitehydro/station_name/fiche"


#É necessário ter o arquivo webdriver num diretório conhecido, ou no diretório corrente: 
driver = webdriver.Chrome()
for station in tqdm.tqdm(network_streams.index[0:2]):
    path = URL.replace("station_name", station[0:-2])
    
    # First we get the URL:
    driver.get(path)
    
    network_streams.loc[station, "area"] = driver.find_element_by_xpath('//*[@id="main"]/main/div/div[2]/div/div[6]/div[4]/span[2]').text
    network_streams.loc[station, "level"] = driver.find_element_by_xpath('//*[@id="main"]/main/div/div[2]/div/div[6]/div[1]/span[2]').text
    
    

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:01<00:00,  1.35it/s]


In [14]:
network_streams.head(2)

,Unnamed: 0,CodeOld,Type,StartDate,EndDate,xcoord,ycoord,area,level,new_code,name
Code,,,,,,,,,,,
A021005050,0,A0210050,STD,1899-12-30 23:50:39,NaN,7.606153,47.556970,35 878 km²,246 m,FR00001,Le Rhin en Suisse [Bâle [Rheinhalle]]
A022020001,1,A0220200,STD,1993-01-24 23:00:00,NaN,7.528982,47.564377,15 km²,277 m,FR00002,L'Augraben à Hégenheim
